<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/35/Tux.svg/280px-Tux.svg.png" width=20% align='right'>

# Podstawy uzytkowania
# systemow komputerowych

Jakub Spiechowicz

Laboratorium 08, Nosniki danych, wyszukiwanie oraz kompresja i archiwizacja plikow

# Filesystem Hierarchy Standard (FHS)

* Jeden system plikow dla wszystkich dyskow fizycznych
* Dyski fizyczne sa *montowane* jako folder w drzewie katalogow systemu plikow

/etc/fstab tablica systemu plikow, plik konfiguracyjny zawierajacy liste nosnikow danych oraz ich punkty montowania na etapie rozruchu systemu

In [1]:
cat /etc/fstab

# UNCONFIGURED FSTAB FOR BASE SYSTEM
155.158.128.81:/users   /home/users     nfs     tcp,port=2049   0       0


Pola konfiguracji fstab

UUID=f2ca55d8-40f9-43c6-80e9-14aa36e915a0 / ext4 errors=remount-ro 0 1

* Identyfikator UUID (Universally Unique Identifier for a Device)
* Punkt montowania
* Typ systemu plikow, ext3, ext4, fat32, ntfs etc.
* Opcje, read-only, etc.
* Liczba reprezentujaca czestosc backupow za pomoca komendy dump
* j.w., ale dotyczy kolejnosci sprawdzania przy uzyciu programu fsck

Lista zamontowanych systemow plikow wedlug wzorca

*Nazwa* on *punkt_montowania* type *typ_systemu_plikow* (*opcje*)

In [6]:
mount | tail -n 7

tmpfs on /run/lock type tmpfs (rw,nosuid,nodev,noexec,relatime,size=5120k)
hugetlbfs on /dev/hugepages type hugetlbfs (rw,relatime)
sunrpc on /run/rpc_pipefs type rpc_pipefs (rw,relatime)
155.158.128.81://users on /home/users type nfs4 (rw,relatime,vers=4.0,rsize=1048576,wsize=1048576,namlen=255,hard,proto=tcp,timeo=600,retrans=2,sec=sys,clientaddr=155.158.128.149,local_lock=none,addr=155.158.128.81)
tmpfs on /run/user/2204 type tmpfs (rw,nosuid,nodev,relatime,size=39602916k,mode=700,uid=2204,gid=2101)
tmpfs on /run/user/2311 type tmpfs (rw,nosuid,nodev,relatime,size=39602916k,mode=700,uid=2311,gid=2100)
tmpfs on /run/user/1000 type tmpfs (rw,nosuid,nodev,relatime,size=39602916k,mode=700,uid=1000,gid=27)


# Montowanie nosnikow danych

Przykladowo dla urzadzenia /dev/hda1 (pierwsza partycja urzadzenia na pierwszym kanale IDE)

su -  
umount /dev/hda1  
mkdir /mnt/hdd  
mount -t ntfs /dev/hda1 /mnt/hdd  
cd /mnt/hdd  
ls  
cd ../  
umount /dev/hda1  

## /dev

* /dev/fd\* dyskietki
* /dev/hd\* dyski IDE (PATA)
  * /dev/hda1 pierwsza partycja na dysku master IDE
  * /dev/hdb3 trzecia partycja na dysku slave IDE
* /dev/lp\* drukarki
* /dev/sd\* dyski SCSI (PATA/SATA, flash, USB, etc.)
  * konwencja nazw jak w przypadku IDE
* /dev/sr\* napedy optyczne (CD/DVD)

Sa tam rowniez linki symboliczne typu /dev/cdrom, /dev/dvd, /dev/floppy etc.

## Tworzenie systemow plikow

Na przykladzie /dev/sdb1

sudo umount /dev/sdb1  
sudo fdisk /dev/sdb

Command (m for help): p
 * p print the partition table

Disk /dev/sdb: 16 MB, 16006656 bytes  
1 heads, 31 sectors/track, 1008 cylinders  
Units = cylinders of 31 * 512 = 15872 bytes
  
Device Boot Start End Blocks Id System  
/dev/sdb1 2 1008 15608+ b W95 FAT32

* l list known partition types
  * aktualny b
  * Linux 83
  
Command (m for help): t

* t change a partition's system id

Selected partition 1  
Hex code (type L to list codes): 83  
Changed system type of partition 1 to 83 (Linux)

Command (m for help): w

* w write table to disk and exit

* *make filesystem*

mkfs

sudo mkfs -t ext4 /dev/sdb1  
sudo mkfs -t vfat /dev/sdb1

# Testowanie i naprawa systemu plikow

* *filesystem check*

sudo fsck /dev/sdb1

Konfiguracja fstab

UUID=f2ca55d8-40f9-43c6-80e9-14aa36e915a0 / ext4 errors=remount-ro 0 1

# Przenoszenie danych miedzy nosnikami

* dd

dd if=wejscie of=wyjscie [bs=rozmiar_bloku [count=liczba_blokow]]

Przyklady 

* kopiowanie dyskow dd if=/dev/sdb of=/dev/sdc
* tworzenie obrazu dysku dd if=/dev/sdb of=flash_drive.img
* tworzenie obrazu CD dd if=/dev/cdrom of=ubuntu.iso
* montowanie obrazu w systemie

mkdir /mnt/iso_image  
mount -t iso9660 -o loop image.iso /mnt/iso_image

# Proste wyszukiwanie plikow

* *locate*

Wyszukiwanie w sciezkach

In [4]:
locate /bin/zip

/usr/bin/zip
/usr/bin/zipcloak
/usr/bin/zipdetails
/usr/bin/zipgrep
/usr/bin/zipinfo
/usr/bin/zipnote
/usr/bin/zipsplit


In [5]:
locate zip | grep bin

/bin/bunzip2
/bin/bzip2
/bin/bzip2recover
/bin/gunzip
/bin/gzip
/usr/bin/funzip
/usr/bin/gpg-zip
/usr/bin/unzip
/usr/bin/unzipsfx
/usr/bin/zip
/usr/bin/zipcloak
/usr/bin/zipdetails
/usr/bin/zipgrep
/usr/bin/zipinfo
/usr/bin/zipnote
/usr/bin/zipsplit
/usr/lib/klibc/bin/gunzip
/usr/lib/klibc/bin/gzip
/usr/lib/sagemath/local/bin/bunzip2
/usr/lib/sagemath/local/bin/bzip2
/usr/lib/sagemath/local/bin/bzip2recover
/usr/lib/sagemath/local/bin/sage-unzip


## Zlozone wyszukiwanie plikow

* *find*

In [17]:
find ~ | tail -n 5

/home/users/jakub.spiechowicz/.sage/ipython-4.2.1/nbextensions
/home/users/jakub.spiechowicz/.sage/R
/home/users/jakub.spiechowicz/.sage/R/Makevars.user
/home/users/jakub.spiechowicz/.sage/db
/home/users/jakub.spiechowicz/PUSK_lab01.ipynb


In [18]:
find ~ | wc -l

99


In [19]:
find ~ -type d | wc -l

52


In [20]:
find ~ -type f | wc -l

47


Typy zawartosci 

* b urzadzenie blokowe
* c urzadzenie strumieniowe
* d katalog
* f plik regularny 
* l link symboliczny

In [24]:
find ~ -type f -name "*.ipynb" -size +10k | wc -l

15


Rozmiar zawartosci

+/- wieksze/mniejsze od

* k kilobajty (1024 bajty)
* M megabajty (1 048 576 bajtow)
* G gigabajty (1 073 741 824 bajtow)

Wybrane testy find, +/-

* -mmin n, ostatnio zmodyfikowano n minut temu
* -newer plik, nowsze niz plik
* -mtime n, ostatnio zmodyfikowane n dni temu
* -empty, puste pliki i katalogi
* -user uzytkownik, pliki i katalogi nalezace do uzytkownika
* -group grupa, pliki i katalogi nalezace do grupy
* -nouser, -nogroup pliki i katalogi nie nalezace do zadnego uzytkownika, grupy
* -perm uprawnienia, pliki lub katalogi z uprawnieniami

In [2]:
find ~ \( -type f -and -perm u=rw,g=r,o=r \) -and -size +10k | wc -l

20


wyrazenie1 -operator wyrazenie2

Operatory logiczne: and (default), or, not

Tabela logiczna find and/or

Rezultat wyrazenia1 | operator | wyrazenie2
------------------- | -------- | ----------  
Prawda | -and | zawsze wyznaczane  
Falsz | -and | nigdy wyznaczane  
True | -or | nigdy wyznaczane  
False | -or | zawsze wyznaczane

In [25]:
find ~ \( -type f -and -perm u=rw,g=r,o=r \) -and -size +10k -and -print | head -n 5

/home/users/jakub.spiechowicz/MPMK_w01_Programowanie_w_Sage.ipynb
/home/users/jakub.spiechowicz/PUSK_lab03.ipynb
/home/users/jakub.spiechowicz/Jupyter_intro.ipynb
/home/users/jakub.spiechowicz/PUSK_lab04.ipynb
/home/users/jakub.spiechowicz/.cache/matplotlib/fontList.cache


Predefiniowane czynnosci find

* -delete usun pasujaca zawartosc
* -ls wykonaj ls dla pasujacej zawartosci
* -print wypisz sciezke pasujacej zawartosci (default)

Kolejnosc ma znaczenie!

Definiowane przez uzytkownika czynnosci find

-exec komenda '{}' ';', np. odpowiednik -delete  
-exec rm '{}' ';'

Tryb interaktywny, -ok

In [27]:
find ~ -type f -name 'PUSK_lab??.ipynb' -exec ls -l '{}' ';'

-rw-r--r-- 1 jakub.spiechowicz staff 31663 Nov  7 12:00 /home/users/jakub.spiechowicz/PUSK_lab03.ipynb
-rw-r--r-- 1 jakub.spiechowicz staff 8226 Nov 10 11:36 /home/users/jakub.spiechowicz/PUSK_lab05.ipynb
-rw-r--r-- 1 jakub.spiechowicz staff 5071 Nov  9 17:28 /home/users/jakub.spiechowicz/PUSK_lab06.ipynb
-rw-r--r-- 1 jakub.spiechowicz staff 11798 Nov 14 12:16 /home/users/jakub.spiechowicz/PUSK_lab04.ipynb
-rw-r--r-- 1 jakub.spiechowicz staff 10110 Nov 29 17:33 /home/users/jakub.spiechowicz/PUSK_lab07.ipynb
-rw-r--r-- 1 jakub.spiechowicz staff 28359 Oct 26 16:40 /home/users/jakub.spiechowicz/PUSK_lab02.ipynb
-rw-r--r-- 1 jakub.spiechowicz staff 17695 Nov 30 20:09 /home/users/jakub.spiechowicz/PUSK_lab08.ipynb
-rw-r--r-- 1 jakub.spiechowicz staff 15709 Oct 13 11:13 /home/users/jakub.spiechowicz/PUSK_lab01.ipynb


In [26]:
find ~ -type f -name 'PUSK_lab??.ipynb' -exec ls -l '{}' +

-rw-r--r-- 1 jakub.spiechowicz staff 15709 Oct 13 11:13 /home/users/jakub.spiechowicz/PUSK_lab01.ipynb
-rw-r--r-- 1 jakub.spiechowicz staff 28359 Oct 26 16:40 /home/users/jakub.spiechowicz/PUSK_lab02.ipynb
-rw-r--r-- 1 jakub.spiechowicz staff 31663 Nov  7 12:00 /home/users/jakub.spiechowicz/PUSK_lab03.ipynb
-rw-r--r-- 1 jakub.spiechowicz staff 11798 Nov 14 12:16 /home/users/jakub.spiechowicz/PUSK_lab04.ipynb
-rw-r--r-- 1 jakub.spiechowicz staff  8226 Nov 10 11:36 /home/users/jakub.spiechowicz/PUSK_lab05.ipynb
-rw-r--r-- 1 jakub.spiechowicz staff  5071 Nov  9 17:28 /home/users/jakub.spiechowicz/PUSK_lab06.ipynb
-rw-r--r-- 1 jakub.spiechowicz staff 10110 Nov 29 17:33 /home/users/jakub.spiechowicz/PUSK_lab07.ipynb
-rw-r--r-- 1 jakub.spiechowicz staff 16419 Nov 30 20:05 /home/users/jakub.spiechowicz/PUSK_lab08.ipynb


## Kompresja plikow

Kompresja pliku = usuwanie z niego nadmiaru informacji

Przyklad

Czarny kwadrat o rozmiarach 100 x 100 pixeli, 24 bity = 3 bajty, 100 x 100 x 3 = 30 000 bajtow,  
alternatywnie 30 000 x 0 => 16 \* 1 bit = 16 bitow = 2 bajty

Algorytmy kompresji

* Bezstratne (png, flac)
* Stratne (jpeg, mp3)

## Gzip/bzip2

gzip/bzip2 pliki

gunzip/bunzip2 pliki

Opcje

* -r kompresja rekursywna
* -t test integralnosci kompresji
* -v tryb gadatliwy
* -liczba stopien kompresji, 1 najszybsza i najmniejsza, 9 najwolniejsza i najwieksza, domyslnie 6

## Archiwizacja plikow

tar pliki

Opcje

* -c utworz archiwum z plikow
* -x rozpakuj archiwum 
* -r dodaj plik na koncu archiwum
* -t wypisz zawartosc archiwum
* -v tryb gadatliwy

Przyklady

* tar -cf playground.tar playground
* tar -tvf playground.tar
* tar -xf playground.tar

## Synchronizacja plikow i katalogow

* *rsync*

rsync opcje zrodlo cel

Zrodlo, cel

* lokalny plik, katalog
* zdalny plik, katalog, user@host:path
* zdalny serwer rsync, rsync://user@host:port/path

Jedno z nich musi byc lokalne.

Przydatne opcje

* -a archiwizacja, tryb rekursywny z zachowaniem atrybutow plikow
* -v tryb gadatliwy

Przyklad

* rsync -av playground foo
* rsync -av --delete --rsh=ssh /etc /home /usr/local zdalny-system:/backup